**Creating experimental trials for Select and Reject conditions across train structures**

This notebook creates experimental trials for 6 experimental protocols from 2 types of relation and 3 train structures. Functions are stored in *trials_v03.py*

- **Relation type** describes the type of relation on baseline pairs during train phase
    - **select_reject**: Baseline pairs trials use other class members stimuli as negative response comparisons, allowing to establish reject relations among other class members.
    - **select_only**: Baseline pairs trials use "dummy" stimuli as negative response comparisons. This prevents the reject relation between sample an other class members stimuli.
- Train Structure: Linear Series (LS), One to Many (OTM) and Many To One (MTO)

Create an experimental protocol following this steps:
1. Create stimuli with selected classes number and members number. This example defines experiments for 5 classes of 7 members each. 
2. Create stimulus pairs according to a specific train structure and relation type. Here, a dictionarie was created with the 6 experimental conditions for each protocol.
3. Create trials from pairs according to pairs subset. Baseline pairs are for training. Transitivity, symmetry and reflexivity pairs are for evaluation.
4. (Optional) A trial and response encoder is added if the agent to train is a Feed-Forward Network, following the paper [Differences of Training Structures on Stimulus Class Formation in Computational Agents](https://www.mdpi.com/2414-4088/7/4/39) This encoding can also be used as a states and actions encoder for an RL agent.

In [1]:
from trials_v03 import *

In [2]:
members_number = 7
classes_number = 5

members_stim, class_stim, stimuli_set, dummy_set = get_stimuli_list(members_n = members_number, classes_n = classes_number)

In [3]:
protocols_dict = {f"{ts}_{relation}": {'train_structure': ts, 'relation_type': relation} 
                  for ts in ["LS", "OTM", "MTO"] for relation in ['select_only', 'select_reject']}
list(protocols_dict.keys())

['LS_select_only',
 'LS_select_reject',
 'OTM_select_only',
 'OTM_select_reject',
 'MTO_select_only',
 'MTO_select_reject']

In [4]:
protocol_names=list(protocols_dict.keys())
prt_nm = 1
print(protocol_names[prt_nm])
protocol = protocols_dict[protocol_names[prt_nm]]
member_pairs_df = create_pair_members(members_stim, train_structure = protocol["train_structure"])
experimental_pairs = create_pairs_classes(member_pairs_df, class_stim)

LS_select_reject


In [5]:
experimental_pairs.sample(15)

,st_sample,st_comparison,pair_subset
63,G4,G4,reflexivity
110,A1,F1,transitivity
78,D4,C4,symmetry
15,D1,E1,baseline
172,D3,F3,transitivity
105,A1,E1,transitivity
207,F3,B3,transitivity
10,C1,D1,baseline
191,E2,C2,transitivity
189,E5,B5,transitivity


In [6]:
baseline_train_trials = create_trials(
    subset_to_trials = "baseline", 
    pairs_dataset_df = experimental_pairs, 
    stimuli_list = stimuli_set, 
    dummy_list = dummy_set, 
    relation_type = protocol['relation_type'])

reflexivity_trials = create_trials(
    subset_to_trials = "reflexivity", 
    pairs_dataset_df = experimental_pairs, 
    stimuli_list = stimuli_set, 
    dummy_list = dummy_set, 
    relation_type = "select_reject")

symmetry_trials = create_trials(
    subset_to_trials = "symmetry", 
    pairs_dataset_df = experimental_pairs, 
    stimuli_list = stimuli_set, 
    dummy_list = dummy_set, 
    relation_type = "select_reject")

transitivity_trials = create_trials(
    subset_to_trials = "transitivity", 
    pairs_dataset_df = experimental_pairs, 
    stimuli_list = stimuli_set, 
    dummy_list = dummy_set, 
    relation_type = "select_reject")

In [7]:
baseline_train_trials.sample(15)

,sample_subset,st_sample,st_comp1,st_comp2,st_comp3,option_answer,st_comparison
1980,baseline,A1,B1,G2,D2,O_1,B1
11156,baseline,A5,G2,E4,B5,O_3,B5
13439,baseline,B1,G3,G2,C1,O_3,C1
29595,baseline,C4,D4,A2,C5,O_1,D4
41316,baseline,D4,E4,B3,A3,O_1,E4
25211,baseline,C2,A5,B5,D2,O_3,D2
49964,baseline,E3,A1,F5,F3,O_3,F3
32320,baseline,C5,B4,D5,A1,O_2,D5
39408,baseline,D3,E3,C4,D5,O_1,E3
16214,baseline,B3,B1,B2,C3,O_3,C3


In [8]:
# This encoding can be used in a loop for validate stability across several experiments avoiding learning related to feature encoding positioning. 
stimuli_dict_encoded = encode_stims (stimuli_set, dummy_set)

In [9]:
baseline_train_values, baseline_train_answers = process_trial_values(baseline_train_trials, stimuli_dict_encoded)
reflexivity_values, reflexivity_answers = process_trial_values(reflexivity_trials, stimuli_dict_encoded)
symmetry_values, symmetry_answers = process_trial_values(symmetry_trials, stimuli_dict_encoded)
transitivity_values, transitivity_answers = process_trial_values(transitivity_trials, stimuli_dict_encoded)

In [10]:
baseline_train_values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [11]:
baseline_train_answers

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       ...,
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])